In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error
import pickle

In [ ]:
df = pd.read_csv('../data/dados_sinteticos_trem.csv', encoding='ISO-8859-1', sep='\t')
df

,#,Horario,TMP,Dia da Semana,Lotacao,Atraso,Estacao,Tipo de Trem
0,0,08:45,25.2,Segunda,39.7,5.6,Barueri,Novo
1,1,00:15,14.3,Segunda,59.0,3.3,Barra Funda,Novo
2,2,01:00,15.3,Segunda,76.2,5.6,Julio Prestes,Novo
3,3,02:30,14.9,Terca,31.5,1.7,Julio Prestes,Novo
4,4,01:00,10.3,Terca,66.4,0.2,Barueri,Antigo
...,...,...,...,...,...,...,...,...
4995,4995,02:15,10.3,Domingo,43.2,4.2,Osasco,Novo
4996,4996,02:00,16.6,Segunda,59.0,4.2,Osasco,Antigo
4997,4997,07:30,12.2,Sabado,45.2,2.7,Osasco,Novo
4998,4998,08:00,20.4,Sabado,77.3,1.0,Barueri,Novo


In [ ]:
def horario_para_minutos(hora):
    h, m = map(int, hora.split(":"))
    return h * 60 + m

df["Horario_min"] = df["Horario"].apply(horario_para_minutos)


X = df[["Horario_min", "Dia da Semana", "Lotacao", "Atraso"]]
y = df["TMP"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
cat_features = ["Dia da Semana"]
num_features = ["Horario_min", "Lotacao", "Atraso"]

preprocessor = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown="ignore"), cat_features),
    ("num", "passthrough", num_features)
])

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("regressor", GradientBoostingRegressor(
        n_estimators=300,      
        max_depth=6,          
        random_state=42,
        verbose=1
    ))
])
pipeline.fit(X_train, y_train)


      Iter       Train Loss   Remaining Time 
         1          14.2502            1.20s
         2          13.1329            1.19s
         3          12.2275            0.99s
         4          11.4892            0.74s
         5          10.8777            1.16s
         6          10.3710            1.16s
         7           9.9495            0.99s
         8           9.6185            0.86s
         9           9.3314            1.14s
        10           9.0922            1.11s
        20           7.9686            0.99s
        30           7.4509            0.93s
        40           7.0645            0.88s
        50           6.6910            0.84s
        60           6.3032            0.81s
        70           5.9845            0.78s
        80           5.6859            0.76s
        90           5.3386            0.73s
       100           5.0810            0.69s
       200           3.2792            0.34s
       300           2.1459            0.00s


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Dia da Semana']),
                                                 ('num', 'passthrough',
                                                  ['Horario_min', 'Lotacao',
                                                   'Atraso'])])),
                ('regressor',
                 GradientBoostingRegressor(max_depth=6, n_estimators=300,
                                           random_state=42, verbose=1))])

In [ ]:
y_pred = pipeline.predict(X_test)


mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f'MAE: {mae:.2f}')
print(f'RMSE: {rmse:.2f}')

MAE: 2.69
RMSE: 3.21


In [ ]:
with open("mlp_previsao_trem.pkl", "wb") as f:
    pickle.dump(pipeline, f)